# CNN em Keras para MNIST (Reprodução do Artigo)

Este notebook implementa, em **TensorFlow/Keras**, a CNN descrita no artigo

> *AN OPTIMIZED CONVOLUTIONAL NEURAL NETWORK FOR HANDWRITTEN DIGITAL RECOGNITION CLASSIFICATION*

Aqui fazemos:

1. Preparação do dataset MNIST  
2. Definição de um modelo CNN com profundidade variável (1, 2 ou 3 blocos Conv+Pool)  
3. Treinamento com **5-fold cross-validation**  
4. Experimentos variando:
   - **taxa de aprendizagem (learning rate)**  
   - **número de épocas (epochs)**  
   - **profundidade da rede (número de blocos Conv+Pool)**  
5. Geração de **gráficos** para análise visual dos resultados.

In [ ]:
# Imports principais
import numpy as np
import matplotlib.pyplot as plt

from tensorflow.keras.datasets import mnist
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense
from tensorflow.keras.optimizers import SGD
from tensorflow.keras.utils import to_categorical
from sklearn.model_selection import KFold
from sklearn.metrics import (
    accuracy_score,
    confusion_matrix,
    classification_report,
    f1_score,
    precision_score,
    recall_score,
)
import time


## 1. Carregando e preparando o dataset MNIST

- Imagens 28x28 em escala de cinza (0–255)  
- Normalizamos para o intervalo **[0, 1]**  
- Adicionamos um eixo de canal para ficar no formato `(28, 28, 1)`  
- Codificamos os rótulos em *one-hot* (10 classes: dígitos de 0 a 9).

In [ ]:
# Carregar MNIST
(x_train, y_train), (x_test, y_test) = mnist.load_data()

# Normalização para [0,1]
x_train = x_train.astype("float32") / 255.0
x_test = x_test.astype("float32") / 255.0

# Adicionar eixo de canal (grayscale)
x_train = np.expand_dims(x_train, axis=-1)
x_test = np.expand_dims(x_test, axis=-1)

# One-hot encoding dos rótulos
num_classes = 10
y_train_cat = to_categorical(y_train, num_classes)
y_test_cat = to_categorical(y_test, num_classes)

x_train.shape, y_train_cat.shape

## 2. Definição do modelo CNN (com profundidade variável)

O artigo trabalha com a ideia de **profundidade** da rede como o número de blocos:

> bloco = `Conv2D → ReLU → MaxPooling2D`

Assim, temos:

- **1 bloco** → rede mais rasa  
- **2 blocos** → rede moderadamente profunda (melhor desempenho no artigo)  
- **3 blocos** → rede mais profunda (piora um pouco o desempenho)  

Após os blocos, usamos:

- `Flatten` para achatar o mapa de características em um vetor  
- `Dense(128, ReLU)` para interpretação das features  
- `Dense(10, Softmax)` para classificação em 10 dígitos.

In [ ]:
def create_cnn_model(num_blocks=1, learning_rate=0.01, momentum=0.9):
    """Cria a CNN conforme a descrição do artigo, com num_blocks blocos Conv+Pool."""
    model = Sequential()

    # Primeiro bloco Conv+Pool (com shape de entrada definido)
    model.add(
        Conv2D(
            filters=32,
            kernel_size=(3, 3),
            activation="relu",
            input_shape=(28, 28, 1),
        )
    )
    model.add(MaxPooling2D(pool_size=(2, 2)))

    # Blocos adicionais (para profundidade 2 ou 3)
    for _ in range(num_blocks - 1):
        model.add(Conv2D(32, (3, 3), activation="relu"))
        model.add(MaxPooling2D(pool_size=(2, 2)))

    # Classificador
    model.add(Flatten())
    model.add(Dense(128, activation="relu"))  # tamanho não especificado no artigo
    model.add(Dense(10, activation="softmax"))

    # Otimizador SGD com momentum (como no artigo)
    optimizer = SGD(learning_rate=learning_rate, momentum=momentum)

    model.compile(
        optimizer=optimizer,
        loss="categorical_crossentropy",
        metrics=["accuracy"],
    )

    return model

# Exemplo: modelo raso com 1 bloco
model_example = create_cnn_model(num_blocks=1)
model_example.summary()

## 3. Funções de treino e avaliação com 5-fold cross-validation

O artigo utiliza **5-fold cross-validation** para:

- Reduzir variância na avaliação  
- Obter **média** e **desvio padrão** da acurácia  

Vamos implementar uma função que:

- Recebe `num_blocks`, `learning_rate`, `epochs`  
- Faz o *split* em 5 folds  
- Treina o modelo em cada fold  
- Retorna acurácia média e desvio padrão em %

In [ ]:
def run_kfold(num_blocks, learning_rate, epochs, batch_size=32, verbose=0, verbose_metrics=True):
    """Executa treinamento com 5-fold cross-validation e calcula métricas detalhadas.

    Retorna um dicionário com:
        - acc_mean, acc_std
        - f1_mean, f1_std
        - prec_mean, prec_std
        - rec_mean, rec_std
        - train_time_sec
        - confusion_matrix (agregada em todas as folds)
        - classification_report (string com precisão/recall/F1 por classe)
    """
    kf = KFold(n_splits=5, shuffle=True, random_state=42)

    accs = []
    f1s = []
    precs = []
    recs = []

    all_y_true = []
    all_y_pred = []

    start_time = time.time()

    for fold_idx, (train_idx, val_idx) in enumerate(kf.split(x_train), start=1):
        x_tr, x_val = x_train[train_idx], x_train[val_idx]
        y_tr, y_val = y_train_cat[train_idx], y_train_cat[val_idx]

        model = create_cnn_model(
            num_blocks=num_blocks,
            learning_rate=learning_rate,
            momentum=0.9,
        )

        model.fit(
            x_tr,
            y_tr,
            epochs=epochs,
            batch_size=batch_size,
            verbose=verbose,
        )

        # Predições no conjunto de validação desta fold
        y_val_prob = model.predict(x_val, verbose=0)
        y_val_pred = np.argmax(y_val_prob, axis=1)
        y_val_true = np.argmax(y_val, axis=1)

        all_y_true.extend(y_val_true.tolist())
        all_y_pred.extend(y_val_pred.tolist())

        # Métricas por fold
        acc = accuracy_score(y_val_true, y_val_pred) * 100.0
        f1_macro = f1_score(y_val_true, y_val_pred, average="macro") * 100.0
        prec_macro = precision_score(
            y_val_true,
            y_val_pred,
            average="macro",
            zero_division=0,
        ) * 100.0
        rec_macro = recall_score(
            y_val_true,
            y_val_pred,
            average="macro",
            zero_division=0,
        ) * 100.0

        accs.append(acc)
        f1s.append(f1_macro)
        precs.append(prec_macro)
        recs.append(rec_macro)

        if verbose_metrics:
            print(
                f"Fold {fold_idx}: Acc={acc:.3f}%, F1_macro={f1_macro:.3f}%, "
                f"Precision_macro={prec_macro:.3f}%, Recall_macro={rec_macro:.3f}%"
            )

    total_time = time.time() - start_time

    all_y_true = np.array(all_y_true)
    all_y_pred = np.array(all_y_pred)

    # Métricas agregadas em todas as folds
    cm = confusion_matrix(all_y_true, all_y_pred)
    report = classification_report(all_y_true, all_y_pred, digits=4)

    summary = {
        "acc_mean": float(np.mean(accs)),
        "acc_std": float(np.std(accs)),
        "f1_mean": float(np.mean(f1s)),
        "f1_std": float(np.std(f1s)),
        "prec_mean": float(np.mean(precs)),
        "prec_std": float(np.std(precs)),
        "rec_mean": float(np.mean(recs)),
        "rec_std": float(np.std(recs)),
        "train_time_sec": float(total_time),
        "confusion_matrix": cm,
        "classification_report": report,
    }

    if verbose_metrics:
        print(f"\nTempo total de treinamento (5 folds): {total_time:.2f} segundos")
        print("\nRelatório de classificação agregado (todas as folds):")
        print(report)
        print("Matriz de confusão agregada (todas as folds):")
        print(cm)

    return summary

## 4. Experimento 1 – Variando a profundidade da rede

Aqui reproduzimos o experimento de **profundidade (model depth)**:

- Testamos **1, 2 e 3 blocos Conv+Pool**  
- Mantemos:
  - `learning_rate = 0.01`  
  - `epochs = 10` (baseline)  

Em seguida, ploteremos um gráfico de barras com a acurácia média de cada configuração.

In [ ]:
depths = [1, 2, 3]

# Listas para guardar as métricas de acurácia (mantendo compatível com os gráficos)
depth_means = []
depth_stds = []

# Listas extras para outras métricas
depth_f1_means = []
depth_f1_stds = []
depth_prec_means = []
depth_prec_stds = []
depth_rec_means = []
depth_rec_stds = []

depth_results = []  # guarda o dicionário completo de cada experimento

for d in depths:
    print(f"\n=== Profundidade: {d} bloco(s) Conv+Pool ===")
    summary = run_kfold(
        num_blocks=d,
        learning_rate=0.01,
        epochs=10,
        verbose=0,
        verbose_metrics=True,
    )

    depth_results.append(summary)

    # Acurácia (para os gráficos já existentes)
    depth_means.append(summary["acc_mean"])
    depth_stds.append(summary["acc_std"])

    # Outras métricas
    depth_f1_means.append(summary["f1_mean"])
    depth_f1_stds.append(summary["f1_std"])
    depth_prec_means.append(summary["prec_mean"])
    depth_prec_stds.append(summary["prec_std"])
    depth_rec_means.append(summary["rec_mean"])
    depth_rec_stds.append(summary["rec_std"])

    print(
        "Resumo final (5-fold): "
        f"Acurácia = {summary['acc_mean']:.3f}% (±{summary['acc_std']:.3f}%), "
        f"F1-macro = {summary['f1_mean']:.3f}% (±{summary['f1_std']:.3f}%), "
        f"Precision-macro = {summary['prec_mean']:.3f}% (±{summary['prec_std']:.3f}%), "
        f"Recall-macro = {summary['rec_mean']:.3f}% (±{summary['rec_std']:.3f}%), "
        f"Tempo treino = {summary['train_time_sec']:.2f} s"
    )

In [ ]:
plt.figure(figsize=(7,5))

x_pos = np.arange(len(depths))

plt.bar(x_pos, depth_means, yerr=depth_stds, capsize=6)

plt.xticks(x_pos, [str(d) for d in depths])
plt.xlabel("Número de blocos Conv+Pool (profundidade)")
plt.ylabel("Acurácia média (%)")
plt.title("Acurácia vs Profundidade da CNN (5-fold)")

# Zoom automático usando zip (funciona com listas)
min_val = min(m - s for m, s in zip(depth_means, depth_stds))
max_val = max(m + s for m, s in zip(depth_means, depth_stds))

plt.ylim(min_val - 0.3, max_val + 0.3)

# Exibir valores no topo das barras
for i, v in enumerate(depth_means):
    plt.text(i, v + 0.05, f"{v:.2f}%", ha='left', fontsize=10)
    
plt.grid(axis='y', linestyle='--', alpha=0.4)

plt.show()

## 5. Experimento 2 – Variando o *learning rate*

Agora avaliamos o impacto do **learning rate**:

- Mantemos:
  - 1 bloco Conv+Pool (modelo baseline)  
  - 10 épocas  
- Testamos os valores usados no artigo:
  - 0.01, 0.05, 0.08, 0.10, 0.15  

Em seguida, ploteremos um gráfico com a acurácia média por valor de learning rate.

In [ ]:
lrs = [0.01, 0.05, 0.08, 0.1, 0.15]

# Listas para acurácia (mantendo compatível com o gráfico existente)
lr_means = []
lr_stds = []

# Listas extras para outras métricas
lr_f1_means = []
lr_f1_stds = []
lr_prec_means = []
lr_prec_stds = []
lr_rec_means = []
lr_rec_stds = []

lr_results = []

for lr in lrs:
    print(f"\n=== Experimento: learning rate = {lr} ===")
    summary = run_kfold(
        num_blocks=1,
        learning_rate=lr,
        epochs=10,
        verbose=0,
        verbose_metrics=True,
    )

    lr_results.append(summary)

    lr_means.append(summary["acc_mean"])
    lr_stds.append(summary["acc_std"])

    lr_f1_means.append(summary["f1_mean"])
    lr_f1_stds.append(summary["f1_std"])
    lr_prec_means.append(summary["prec_mean"])
    lr_prec_stds.append(summary["prec_std"])
    lr_rec_means.append(summary["rec_mean"])
    lr_rec_stds.append(summary["rec_std"])

    print(
        "Resumo final (5-fold): "
        f"Acurácia = {summary['acc_mean']:.3f}% (±{summary['acc_std']:.3f}%), "
        f"F1-macro = {summary['f1_mean']:.3f}% (±{summary['f1_std']:.3f}%), "
        f"Precision-macro = {summary['prec_mean']:.3f}% (±{summary['prec_std']:.3f}%), "
        f"Recall-macro = {summary['rec_mean']:.3f}% (±{summary['rec_std']:.3f}%), "
        f"Tempo treino = {summary['train_time_sec']:.2f} s"
    )

In [ ]:
# Gráfico: acurácia média x learning rate
plt.figure()
x_pos = np.arange(len(lrs))
plt.plot(x_pos, lr_means, marker="o")
plt.xticks(x_pos, [str(lr) for lr in lrs])
plt.xlabel("Learning rate")
plt.ylabel("Acurácia média (%)")
plt.title("Acurácia vs Learning rate (5-fold)")
plt.grid(True)
plt.show()

## 6. Experimento 3 – Variando o número de épocas

Agora estudamos o efeito de **mais épocas de treino** na performance:

- Mantemos:
  - 1 bloco Conv+Pool  
  - learning rate = 0.01  
- Testamos:
  - 10, 15, 20, 25, 30 épocas  

Em seguida, veremos o gráfico de acurácia média por número de épocas.

In [ ]:
epochs_list = [10, 15, 20, 25, 30]

# Listas de acurácia (compatíveis com o gráfico existente)
ep_means = []
ep_stds = []

# Listas extras para outras métricas
ep_f1_means = []
ep_f1_stds = []
ep_prec_means = []
ep_prec_stds = []
ep_rec_means = []
ep_rec_stds = []

ep_results = []

for ep in epochs_list:
    print(f"\n=== Experimento: épocas = {ep} ===")
    summary = run_kfold(
        num_blocks=1,
        learning_rate=0.01,
        epochs=ep,
        verbose=0,
        verbose_metrics=True,
    )

    ep_results.append(summary)

    ep_means.append(summary["acc_mean"])
    ep_stds.append(summary["acc_std"])

    ep_f1_means.append(summary["f1_mean"])
    ep_f1_stds.append(summary["f1_std"])
    ep_prec_means.append(summary["prec_mean"])
    ep_prec_stds.append(summary["prec_std"])
    ep_rec_means.append(summary["rec_mean"])
    ep_rec_stds.append(summary["rec_std"])

    print(
        "Resumo final (5-fold): "
        f"Acurácia = {summary['acc_mean']:.3f}% (±{summary['acc_std']:.3f}%), "
        f"F1-macro = {summary['f1_mean']:.3f}% (±{summary['f1_std']:.3f}%), "
        f"Precision-macro = {summary['prec_mean']:.3f}% (±{summary['prec_std']:.3f}%), "
        f"Recall-macro = {summary['rec_mean']:.3f}% (±{summary['rec_std']:.3f}%), "
        f"Tempo treino = {summary['train_time_sec']:.2f} s"
    )

In [ ]:
# Gráfico: acurácia média x número de épocas
plt.figure()
x_pos = np.arange(len(epochs_list))
plt.plot(x_pos, ep_means, marker="o")
plt.xticks(x_pos, [str(ep) for ep in epochs_list])
plt.xlabel("Número de épocas")
plt.ylabel("Acurácia média (%)")
plt.title("Acurácia vs Número de épocas (5-fold)")
plt.grid(True)
plt.show()

## 7. Conclusões

A partir dos experimentos, pode-se comparar com os resultados do artigo:

- **Profundidade**: geralmente, 2 blocos Conv+Pool tendem a performar melhor que 1 ou 3 blocos.  
- **Learning rate**: valores muito altos (ex: 0.1, 0.15) degradam a acurácia; algo próximo a **0.01** costuma ser mais estável.  
- **Número de épocas**: aumentar as épocas nem sempre melhora indefinidamente; há um ponto de retorno decrescente, e o custo computacional cresce.  

É possivel ajustar os hiperparâmetros e repetir os blocos de código dos experimentos para novas análises.